In [13]:
import pandas as pd 
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [2]:
chemin_excel_blessures = "../DataSets/BlessuresTennis_Causes.xlsx"
df = pd.read_excel(chemin_excel_blessures)

In [3]:
# Nettoyage des données
def fix_separators(df):
    corrections = {}
    for column in df.columns:
        corrections[column] = []
        for i, value in enumerate(df[column]):
            if pd.notnull(value) and '; ' in str(value):
                parts = str(value).split('; ')
                if len(parts) > 1:
                    df.at[i, column] = '; '.join(part.strip() for part in parts)
                    corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
            elif pd.notnull(value) and ';' in str(value):
                df.at[i, column] = '; '.join(part.strip() for part in str(value).split(';') if part.strip() != '')
                corrections[column].append(f"Ligne {i+1}: Séparateur corrigé")
    return df, corrections

df, corrections = fix_separators(df[['ID', 'CategBlessure', 'TypeBlessure', 'Blessure', 'Symptomes_Blessures', 'Traitement_Blessures', 'Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions', 'Causes_Blessure', 'Zone_Blessures', 'Description_Zone']])
print("Séparateurs corrigés :")
for column, correction in corrections.items():
    if correction:
        print(f"Colonne {column}:")
        for item in correction:
            print(f"- {item}")
        print('\n')

Séparateurs corrigés :
Colonne Blessure:
- Ligne 4: Séparateur corrigé
- Ligne 21: Séparateur corrigé
- Ligne 22: Séparateur corrigé


Colonne Symptomes_Blessures:
- Ligne 1: Séparateur corrigé
- Ligne 2: Séparateur corrigé
- Ligne 3: Séparateur corrigé
- Ligne 4: Séparateur corrigé
- Ligne 5: Séparateur corrigé
- Ligne 7: Séparateur corrigé
- Ligne 8: Séparateur corrigé
- Ligne 9: Séparateur corrigé
- Ligne 10: Séparateur corrigé
- Ligne 15: Séparateur corrigé
- Ligne 21: Séparateur corrigé
- Ligne 22: Séparateur corrigé
- Ligne 24: Séparateur corrigé
- Ligne 31: Séparateur corrigé
- Ligne 32: Séparateur corrigé
- Ligne 34: Séparateur corrigé
- Ligne 38: Séparateur corrigé


Colonne Traitement_Blessures:
- Ligne 1: Séparateur corrigé
- Ligne 6: Séparateur corrigé
- Ligne 12: Séparateur corrigé
- Ligne 18: Séparateur corrigé
- Ligne 20: Séparateur corrigé
- Ligne 24: Séparateur corrigé
- Ligne 25: Séparateur corrigé
- Ligne 26: Séparateur corrigé
- Ligne 29: Séparateur corrigé
- Ligne 

In [4]:
def extract_float(value):
    matches = re.findall(r'\d+\.\d+', str(value)) 
    if matches:
        return float(matches[0])
    else:
        return None

In [5]:
# Remplacer les virgules par des points dans les valeurs de la colonne 'Prix_Prescriptions (en € )'
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].str.replace(',', '.')
df['Prix_Prescriptions (en € )'] = df['Prix_Prescriptions (en € )'].apply(extract_float)

df = df.dropna(subset=['Prix_Prescriptions (en € )'])

In [6]:
columns_to_convert = ['Prescriptions_Blessures', 'Prix_Prescriptions (en € )', 'Images_Prescriptions', 'Descriptifs_Prescriptions', 'LienAchat_Prescriptions']
df = df.dropna(subset=columns_to_convert, how='all')
df.head()

,ID,CategBlessure,TypeBlessure,Blessure,Symptomes_Blessures,Traitement_Blessures,Prescriptions_Blessures,Prix_Prescriptions (en € ),Images_Prescriptions,Descriptifs_Prescriptions,LienAchat_Prescriptions,Causes_Blessure,Zone_Blessures,Description_Zone
0,1,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de Quervain,Inflammation douloureuse des tendons près du p...,infiltrations locales de cortisone; réeduction...,Orthèse poignet-pouce tendinite NUIT; Orthèse ...,55.31,https://epitact.twic.pics/prod/2024-02/mea-que...,Soulage les douleurs liées à la tendinite De Q...,https://epitact.fr/quervimmo-orthese-immobilis...,Utilisation excessive du poignet et du pouce; ...,Zone de transition,La zone intermédiaire entre le fond de court e...
1,2,Blessures aux membres supérieurs,Douleur à la main et au poignet,ténosynovite de l’extenseur ulnaire du carpe,Inflammation douloureuse des tendons près du c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Surutilisation du poignet; Mauvaise posture; M...,Zone médiane gauche; Zone médiane droite,Les zones près du centre du terrain où les jou...
2,3,Blessures aux membres supérieurs,Douleur à la main et au poignet,entorse du ligament carpien,Blessure traumatique au ligament du poignet; c...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Torsion excessive du poignet; Chute/Impact direct,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
4,5,Blessures aux membres supérieurs,Douleur à la main et au poignet,lésions du ligament annulaire du carpe,Blessures au ligament du poignet; causant doul...,NaN,Orthèse poignet canal carpien JOUR; Orthèse po...,23.99,https://epitact.twic.pics/prod/2024-02/mea-car...,Soulage les douleurs liées au syndrome du cana...,https://epitact.fr/carp-activ?list=Article%20T...,Utilisation excessive; Mouvements répétitifs,Coins inférieur gauche; Coins supérieur gauche,Les coins du fond de court près du filet où le...
14,16,Blessures aux membres supérieurs,Douleur au dos et au ventre,Les tensions musculaires abdominales,Inconfort ou douleur dans les muscles de l'abd...,NaN,Posturostrap™ Dos,79.99,https://epitact.twic.pics/prod/2024-02/posturo...,Prévient le mal de dos lors des activités quot...,https://epitact.fr/correcteur-de-posture-postu...,Surutilisation; Mouvements brusques; Echauffem...,Coins inférieur droit; Coins supérieur droit,Les coins du fond de court où les joueurs subi...


In [14]:
df_blessures = pd.read_excel('BlessuresTennis_Causes.xlsx')
df_blessures['Traitement_Blessures'] = df_blessures['Traitement_Blessures'].fillna('')
for column in df_blessures.columns:
    if df_blessures[column].dtype == 'object':
        df_blessures[column] = df_blessures[column].astype(str)
label_encoders = {}
for column in df_blessures.columns:
    if df_blessures[column].dtype == 'object':
        le = LabelEncoder()
        df_blessures[column] = le.fit_transform(df_blessures[column])
        label_encoders[column] = le
X = df_blessures.drop(columns=['TypeBlessure'])
y = df_blessures['TypeBlessure']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.625


c:\Users\sonia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.625
